In [9]:
!python -m pip install -U sentence-transformers
!python -m pip install texthero
!python -m pip install zeroshot_topics

  Using cached texthero-1.1.0-py3-none-any.whl (24 kB)
  Using cached pandas-1.4.0-cp38-cp38-win_amd64.whl (10.6 MB)
  Using cached gensim-3.8.3-cp38-cp38-win_amd64.whl (24.2 MB)


In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\bishw\Downloads\complaint_data.csv")

df.head()

df1 = df.loc[:,['Complaint ID','Consumer complaint narrative']].copy(deep = True)

df1.head()

df1.dropna(inplace = True)



In [2]:
import texthero as hero
import nltk
nltk.download('punkt')


dfsample = df1.sample(5000)

from nltk.tokenize import sent_tokenize
s = dfsample["Consumer complaint narrative"].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()


s = pd.DataFrame(s, columns = ['Broken_Survey'])
s.head()

s['Broken_Survey'] = hero.clean(s['Broken_Survey'])






[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bishw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bishw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = s.Broken_Survey.to_list()
sentence_embeddings = model.encode(sentences, show_progress_bar=True)

Batches:   0%|          | 0/1526 [00:00<?, ?it/s]

c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [5]:
from sentence_transformers import util
import time

print("Start clustering")
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(sentence_embeddings, min_community_size=25, threshold=0.75)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))



#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", sentences[sentence_id])

cluster_map = {}
for i, cluster in enumerate(clusters):
    sent_list = []
    for sentence_id in cluster:
        sent_list.append(sentences[sentence_id])
    cluster_map[i] = sent_list


cluster_map_1 = {}
for k,v in cluster_map.items():
  s = '.'.join(v)
  cluster_map_1[k] = s

cluster_map_1

Start clustering
Clustering done after 26.35 sec

Cluster 1, #560 Elements 
	 
	 
	 
	 ...
	 
	 
	 

Cluster 2, #423 Elements 
	 xxxx xx xx xxxx
	 xxxx xx xx xxxx
	 xxxx xx xx xxxx
	 ...
	 xxxx xxxx xxxx times xxxx xxxx
	 another xxxx xx xx
	 xxxx xxxx help

Cluster 3, #149 Elements 
	 thank
	 thank
	 thank
	 ...
	 thanks
	 thanks
	 thanks

Cluster 4, #149 Elements 
	 payments made xxxx xxxx xxxx
	 made payments xxxx
	 made xxxx payment xxxx xxxx xxxx payment xxxx xxxx
	 ...
	 payment xx xx xxxx would make two payments xxxx
	 xx xx xxxx made arrangements xxxx xxxx take xxxx dollars monthly account electronic check debt xxxx paid
	 past due balance payments made advised xxxx xxxx xxxx

Cluster 5, #129 Elements 
	 please help
	 please help
	 please help
	 ...
	 really need help
	 thank help
	 thank help

Cluster 6, #129 Elements 
	 need removed credit report
	 would like removed credit report
	 please remove credit report
	 ...
	 since bureus valide debt intirety requested want deleted r

{0: '...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................',
 1: 'xxxx xx xx xxxx.xxxx xx xx xxxx.xxxx xx xx xxxx.xxxx xx xx xxxx.xxxx xx xx xxxx.xxxx xx xx xxxx.xxxx xx xx xxxx.xx xx xxxx xxxx xxxx.xx xx xxxx xxxx xxxx.xx xx xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxxx.xxxx xxxx xxx

In [6]:
from zeroshot_topics import ZeroShotTopicFinder
import nltk
nltk.download('wordnet')
zsmodel = ZeroShotTopicFinder()


c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bishw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
zsmodel.find_topic(cluster_map_1.get(8))

c:\users\bishw\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


['Crime', 'Informing']

In [34]:
cluster_map_1.get(8)

'shocked reviewed credit report found late payments dates creditor xxxx xxxx account xxxx never late days months xxxx xxxx.shocked reviewed credit report found late payment dates creditor xxxx xxxx account xxxx never late days late months xx xx xxxx xx xx xxxx.shocked reviewed credit report found days late payment dates xx xx xx xx xxxx.shocked reviewing credit report found late payments dates days late xx xx xxxx xx xx xxxx bureaus sure happened.shocked reviewed credit report found days late payment dates xx xx xx xx xxxx xx xx xx xx xx xx xx xx xx xx xxxx xx xx xx xx.account number xxxx shocked reviewed credit report found late payment dates days past due xxxx sure happened believe made payments received statements.shocked reviewed credit report found late payments following dates xxxx xxxx xxxx xxxx xxxx xxxx xxxx xxxx xxxx xxxx sure happened.shocked reviewed credit report found late payment dates xx xx days late.shocked reviewing credit report found late payment date days late xx x